In [45]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

from sklearn.model_selection import GridSearchCV, cross_validate, RandomizedSearchCV, validation_curve
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, log_loss
import warnings
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [68]:
pd.set_option('display.max_column' , None)
warnings.simplefilter(action='ignore',category=Warning)
pd.options.display.float_format = '{:,.3f}'.format

In [7]:
df = pd.read_csv("../../datasets/Churn/Telco-Customer-Churn.csv")

In [4]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.850,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.950,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.850,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.300,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.700,151.65,Yes


In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
SeniorCitizen,"7,043.000",0.162,0.369,0.000,0.000,0.000,0.000,1.000
tenure,"7,043.000",32.371,24.559,0.000,9.000,29.000,55.000,72.000
MonthlyCharges,"7,043.000",64.762,30.090,18.250,35.500,70.350,89.850,118.750


In [10]:
def grab_col_names(data_frame, cat_th=25 , car_th=30): 
    cat_cols = [col for col in data_frame.columns if data_frame[col].dtype == "O"]
    num_but_cat = [col for col in data_frame.columns if data_frame[col].nunique() < car_th and data_frame[col].dtype != "O"]
    cat_but_car = [col for col in data_frame.columns if data_frame[col].nunique() > car_th and data_frame[col].dtype == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]
    num_cols = [col for col in data_frame.columns if data_frame[col].dtype != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {data_frame.shape[0]}")
    print(f"Variables: {data_frame.shape[1]}")
    print(f"cat_cols: {len(cat_cols)}")
    print(f"cat_but_car: {len(cat_but_car)}")
    print(f"num_cols: {len(num_cols)}")
    print(f"num_but_car: {len(num_but_cat)}")
    return num_cols,cat_cols,cat_but_car

def missing_values_table(data_frame, na_name=False):
    na_columns = [col for col in data_frame.columns if data_frame[col].isnull().sum() > 0]

    n_miss = data_frame[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (data_frame[na_columns].isnull().sum() / data_frame.shape[0] * 100)
    missing_df = pd.concat([n_miss, np.round(ratio,2)],axis=1, keys=["n_miss" , "ratio"])
    print(missing_df,end="\n")

    if na_name:
        return na_columns

def locate_uneven_cols(dataframe, col):
    total_values = dataframe[col].value_counts().sum()
    val_counts = dataframe[col].value_counts() 
    for count in val_counts:
        percentage = (count / total_values) * 100
        if percentage > 95:
            return True
    return False

In [11]:
num_cols , cat_cols, cat_but_car = grab_col_names(df)

Observations: 7043
Variables: 21
cat_cols: 17
cat_but_car: 2
num_cols: 2
num_but_car: 1


In [12]:
cat_cols

['gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'Churn',
 'SeniorCitizen']

In [13]:
bool_cols = [col for col in df.columns if (df[col].dtype == "O") & (df[col].nunique() == 2)]
bool_cols

['gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'PaperlessBilling',
 'Churn']

In [14]:
missing_values_table(df)

Empty DataFrame
Columns: [n_miss, ratio]
Index: []


In [15]:
for col in df.columns:
    if df[col].nunique() == 2:
        print("#######",end="\n\n")
        print(f"{col} = boolean")
    else:
        print("#######",end="\n\n")
        
        print(f"{col} = NOT BOOLEAN , count : {df[col].nunique()}")

#######

customerID = NOT BOOLEAN , count : 7043
#######

gender = boolean
#######

SeniorCitizen = boolean
#######

Partner = boolean
#######

Dependents = boolean
#######

tenure = NOT BOOLEAN , count : 73
#######

PhoneService = boolean
#######

MultipleLines = NOT BOOLEAN , count : 3
#######

InternetService = NOT BOOLEAN , count : 3
#######

OnlineSecurity = NOT BOOLEAN , count : 3
#######

OnlineBackup = NOT BOOLEAN , count : 3
#######

DeviceProtection = NOT BOOLEAN , count : 3
#######

TechSupport = NOT BOOLEAN , count : 3
#######

StreamingTV = NOT BOOLEAN , count : 3
#######

StreamingMovies = NOT BOOLEAN , count : 3
#######

Contract = NOT BOOLEAN , count : 3
#######

PaperlessBilling = boolean
#######

PaymentMethod = NOT BOOLEAN , count : 4
#######

MonthlyCharges = NOT BOOLEAN , count : 1585
#######

TotalCharges = NOT BOOLEAN , count : 6531
#######

Churn = boolean


In [45]:
df["OnlineSecurity"].value_counts()

OnlineSecurity
No                     3498
Yes                    2019
No internet service    1526
Name: count, dtype: int64

In [41]:
uneven_cols = []
for col in df.columns:
    bool = locate_uneven_cols(df,col)
    if bool:
        uneven_cols.append(col)

In [42]:
uneven_cols

[]

In [46]:
df["Churn"].value_counts()

Churn
No     5174
Yes    1869
Name: count, dtype: int64

In [48]:
for col in df.columns:
    if df[col].nunique() < 10:
        print("########################", end="\n\n\n")
        print(f"{col} value counts : {df[col].value_counts()} ")

########################


gender value counts : gender
Male      3555
Female    3488
Name: count, dtype: int64 
########################


SeniorCitizen value counts : SeniorCitizen
0    5901
1    1142
Name: count, dtype: int64 
########################


Partner value counts : Partner
No     3641
Yes    3402
Name: count, dtype: int64 
########################


Dependents value counts : Dependents
No     4933
Yes    2110
Name: count, dtype: int64 
########################


PhoneService value counts : PhoneService
Yes    6361
No      682
Name: count, dtype: int64 
########################


MultipleLines value counts : MultipleLines
No                  3390
Yes                 2971
No phone service     682
Name: count, dtype: int64 
########################


InternetService value counts : InternetService
Fiber optic    3096
DSL            2421
No             1526
Name: count, dtype: int64 
########################


OnlineSecurity value counts : OnlineSecurity
No                    

In [57]:
"No internet service" in df["StreamingMovies"].unique()

True

In [137]:
for col in df.columns:
    if "No internet service" in df[col].unique():
        df[col] = df[col].replace({"No internet service":"No"})  

In [60]:
for col in df.columns:
    if df[col].nunique() < 10:
        print("########################", end="\n\n\n")
        print(f"{col} value counts : {df[col].value_counts()} ")

########################


gender value counts : gender
Male      3555
Female    3488
Name: count, dtype: int64 
########################


SeniorCitizen value counts : SeniorCitizen
0    5901
1    1142
Name: count, dtype: int64 
########################


Partner value counts : Partner
No     3641
Yes    3402
Name: count, dtype: int64 
########################


Dependents value counts : Dependents
No     4933
Yes    2110
Name: count, dtype: int64 
########################


PhoneService value counts : PhoneService
Yes    6361
No      682
Name: count, dtype: int64 
########################


MultipleLines value counts : MultipleLines
No                  3390
Yes                 2971
No phone service     682
Name: count, dtype: int64 
########################


InternetService value counts : InternetService
Fiber optic    3096
DSL            2421
No             1526
Name: count, dtype: int64 
########################


OnlineSecurity value counts : OnlineSecurity
No     5024
Yes    201

In [138]:
for col in df.columns:
    if "No phone service" in df[col].unique():
        df[col] = df[col].replace({"No phone service":"No"})  

In [146]:
df.drop(["customerID"],axis=1, inplace=True)

In [148]:
num_col, cat_col, cat_but_car = grab_col_names(df)

Observations: 7043
Variables: 20
cat_cols: 17
cat_but_car: 0
num_cols: 3
num_but_car: 1


In [147]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [128]:
num_cols

['tenure', 'MonthlyCharges', 'TotalCharges']

In [171]:
df["SeniorCitizen"] = df["SeniorCitizen"].map({False: "No", True: "Yes"})

In [120]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.850,29.850,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.950,"1,889.500",No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.850,108.150,Yes
3,Male,0,No,No,45,No,No,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.300,"1,840.750",No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.700,151.650,Yes


In [111]:
label_encoder = LabelEncoder()

In [174]:
def label_encode(le, df, col, out=False):
    df[col] = le.fit_transform(df[col])
    if out:
        print(f"{col} label encoded")

In [114]:
def one_hot_encode(data_frame, categorical_cols, drop_first=True):
    data_frame = pd.get_dummies(data_frame, columns=categorical_cols,drop_first=drop_first)
    return data_frame

In [157]:
for col in df.columns:
    print(f"{col} : {df[col].nunique()}")

gender : 2
SeniorCitizen : 2
Partner : 2
Dependents : 2
tenure : 73
PhoneService : 2
MultipleLines : 2
InternetService : 3
OnlineSecurity : 2
OnlineBackup : 2
DeviceProtection : 2
TechSupport : 2
StreamingTV : 2
StreamingMovies : 2
Contract : 3
PaperlessBilling : 2
PaymentMethod : 4
MonthlyCharges : 1585
TotalCharges : 6530
Churn : 2


In [164]:
ohe_cols = []
label_encode_cols = []
numeric_cols = []
for col in df.columns:
    if df[col].nunique() < 3:
       label_encode_cols.append(col)
    elif df[col].nunique() > 2 and df[col].nunique() < 11:
        ohe_cols.append(col)
    else:
        numeric_cols.append(col)

['InternetService', 'Contract', 'PaymentMethod']
['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn']
['tenure', 'MonthlyCharges', 'TotalCharges']


In [165]:
ohe_cols

['InternetService', 'Contract', 'PaymentMethod']

In [169]:
label_encode_cols

['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'PaperlessBilling']

In [167]:
numeric_cols

['tenure', 'MonthlyCharges', 'TotalCharges']

In [172]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,No,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.850,29.850,No
1,Male,No,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.950,"1,889.500",No
2,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.850,108.150,Yes
3,Male,No,No,No,45,No,No,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.300,"1,840.750",No
4,Female,No,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.700,151.650,Yes


In [173]:
dff = df.copy()

In [175]:
for col in dff.columns:
    label_encode(label_encoder, dff, col, True)

gender label encoded
SeniorCitizen label encoded
Partner label encoded
Dependents label encoded
tenure label encoded
PhoneService label encoded
MultipleLines label encoded
InternetService label encoded
OnlineSecurity label encoded
OnlineBackup label encoded
DeviceProtection label encoded
TechSupport label encoded
StreamingTV label encoded
StreamingMovies label encoded
Contract label encoded
PaperlessBilling label encoded
PaymentMethod label encoded
MonthlyCharges label encoded
TotalCharges label encoded
Churn label encoded


In [212]:
dff

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,InternetService_1,InternetService_2,Contract_1,Contract_2,PaymentMethod_1,PaymentMethod_2,PaymentMethod_3
0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,142,74,0,False,False,False,False,False,True,False
1,1,0,0,0,34,1,0,1,0,1,0,0,0,0,498,3624,0,False,False,True,False,False,False,True
2,1,0,0,0,2,1,0,1,1,0,0,0,0,1,436,536,1,False,False,False,False,False,False,True
3,1,0,0,0,45,0,0,1,0,1,1,0,0,0,266,3570,0,False,False,True,False,False,False,False
4,0,0,0,0,2,1,0,0,0,0,0,0,0,1,729,674,1,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,0,1,1,24,1,1,1,0,1,1,1,1,1,991,3700,0,False,False,True,False,False,False,True
7039,0,0,1,1,72,1,1,0,1,1,0,1,1,1,1340,6304,0,True,False,True,False,True,False,False
7040,0,0,1,1,11,0,0,1,0,0,0,0,0,1,137,1265,0,False,False,False,False,False,True,False
7041,1,1,1,0,4,1,1,0,0,0,0,0,0,1,795,1157,1,True,False,False,False,False,False,True


In [203]:
dff = one_hot_encode(dff, ohe_cols)

In [179]:
def check_outliers(data_frame, col_name):
    up_limit,low_limit = outliers_treshold(data_frame,col_name)
    if data_frame[(data_frame[col_name] > up_limit) | (data_frame[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False

def outliers_treshold(data_frame, col_name, q1=0.25 ,q3=0.75):
    quartile1 = data_frame[col_name].quantile(q1)
    quartile3 = data_frame[col_name].quantile(q3)
    interquartile_range = quartile3-quartile1
    up_limit = quartile3 + 1.5 * interquartile_range
    low_limit = quartile1 - 1.5 * interquartile_range
    return up_limit, low_limit

def grab_outliers(data_frame,col_name,index=False):
    up,low = outliers_treshold(data_frame,col_name)
    # if data_frame[((data_frame[col_name] < low) | (data_frame[col_name] > up))].shape[0] > 10:
    #     print(data_frame[((data_frame[col_name] < low) | (data_frame[col_name] > up))].head())
    # else:
    #     print(data_frame[((data_frame[col_name] < low) | (data_frame[col_name] > up))])
    if index:
        outlier_index = data_frame[((data_frame[col_name] < low) | (data_frame[col_name] > up))].index
        return outlier_index

In [181]:
outliers = []
for col in dff.columns:
    if check_outliers(dff,col):
        print(col)
        outliers.append(col)

SeniorCitizen
PhoneService


In [184]:
ss = StandardScaler()

In [204]:
df3 = dff.copy()

In [205]:
X = df3.drop(["Churn"], axis=1)
y = df3["Churn"]

In [206]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [207]:
X_train_scaled = ss.fit_transform(X_train)

In [208]:
X_test_scaled = ss.transform(X_test)

In [209]:
X_train_scaled.shape

(5634, 23)

In [25]:
def cv(model,X,y,cv=10,scoring=["accuracy", "f1","roc_auc"]):
    cv_results = cross_validate(model, X, y, cv=cv, scoring=scoring, return_train_score=False)
    print(f"accuracy: {cv_results['test_accuracy'].mean()}")
    print(f"f1: {cv_results['test_f1'].mean()}")
    print(f"roc_auc: {cv_results['test_roc_auc'].mean()}")

# Logistic regression Standart scaled

In [230]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

log_reg = LogisticRegression()
log_reg.fit(X_train_scaled, y_train)

predictions = log_reg.predict(X_test_scaled)
print(f"Accuracy: {accuracy_score(y_test, predictions)}")


Accuracy: 0.8126330731014905


In [299]:
cv(log_reg, X_train_scaled, y_train)


accuracy: 0.8049270624692945
f1: 0.5799320966555255
roc_auc: 0.8432064243087769


In [233]:
rs = RobustScaler()

X_train_robust_scaled = rs.fit_transform(X_train)
X_test_robust_scaled = rs.fit_transform(X_test)

# Logisctic Regression robust scaled

In [308]:
log_reg = LogisticRegression()
log_reg.fit(X_train_robust_scaled, y_train)

predictions = log_reg.predict(X_test_robust_scaled)
print(f"Accuracy: {accuracy_score(y_test, predictions)}")

Accuracy: 0.8133427963094393


In [309]:
cv(log_reg, X_train_robust_scaled, y_train)

accuracy: 0.8058135872919895
f1: 0.5829407515490971
roc_auc: 0.8432737672165447


In [310]:
cv(log_reg,X_test,y_test)

accuracy: 0.8097720364741642
f1: 0.5978153189368652
roc_auc: 0.8564273910914176


# Smote sampling

In [253]:
sm = SMOTE(random_state=42)

X_train_res, y_train_res = sm.fit_resample(X_train_robust_scaled, y_train)

In [311]:
log_reg = LogisticRegression()
log_reg.fit(X_train_res, y_train_res)

predictions = log_reg.predict(X_test)

cv(log_reg,X_train_res,y_train_res)

accuracy: 0.7689715752764488
f1: 0.7737419691158586
roc_auc: 0.8515509232064827


In [342]:
cv(log_reg,X_test_scaled,y_test)


accuracy: 0.8104863221884498
f1: 0.6064649621104454
roc_auc: 0.8563320193941042


# Random forest 

In [313]:
rf_model = RandomForestClassifier(random_state=42)

model.fit(X_train_res, y_train_res)

cv(rf_model, X_train_res, y_train_res)

accuracy: 0.8591328882112752
f1: 0.8565592726059782
roc_auc: 0.9309308436682067


In [343]:
# predictions = log_reg.predict(X_test)
# pred_proba = log_reg.predict_proba(X_test)[:,1]

# print(f"Accuracy: {accuracy_score(y_test, predictions)}")
# print(f"f1: {f1_score(y_test, predictions)}")
# print(f"roc_auc: {roc_auc_score(y_test, pred_proba)}")

cv(rf_model, X_test_scaled, y_test)

accuracy: 0.7827963525835866
f1: 0.5422854114278304
roc_auc: 0.829800242638771


In [363]:
search_params = {"max_depth":[7,8,9,10],
                "max_features":[3,5,7,"auto"],
                "min_samples_split":[2,5,8,15,20],
                "n_estimators":[200,700,800,100]}

In [364]:
best_params = GridSearchCV(rf_model, search_params, cv=5, verbose=True, n_jobs=-1).fit(X_train_res,y_train_res)

Fitting 5 folds for each of 320 candidates, totalling 1600 fits


In [361]:
rf_model.set_params(**best_params.best_params_)

RandomForestClassifier(max_depth=7, max_features=5, min_samples_split=8,
                       n_estimators=200, random_state=42)

In [366]:
rf_model.set_params(**best_params.best_params_)


RandomForestClassifier(max_depth=10, max_features=5, random_state=42)

In [367]:
cv(rf_model, X_test, y_test)


accuracy: 0.7927406281661601
f1: 0.5665033720231631
roc_auc: 0.8380964847654935


In [362]:
cv(rf_model, X_test, y_test)

accuracy: 0.8083535967578521
f1: 0.5864491173357537
roc_auc: 0.8479456044031926


In [344]:
from sklearn.svm import SVC

svm_clf = SVC(probability=True)
svm_clf.fit(X_train_res, y_train_res)

cv(svm_clf, X_test_scaled, y_test)

accuracy: 0.7970111448834853
f1: 0.5610291364660578
roc_auc: 0.8387464185816256


In [341]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'degree': [2, 3, 4]  # Only used for 'poly' kernel
}

# Grid search
grid_search = GridSearchCV(svm_clf, param_grid, refit=True, verbose=2, cv=5)
grid_search.fit(X_train_res, y_train_res)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV] END ...............C=0.1, degree=2, gamma=1, kernel=rbf; total time=   8.8s
[CV] END ...............C=0.1, degree=2, gamma=1, kernel=rbf; total time=   8.8s
[CV] END ...............C=0.1, degree=2, gamma=1, kernel=rbf; total time=   9.3s
[CV] END ...............C=0.1, degree=2, gamma=1, kernel=rbf; total time=   8.9s
[CV] END ...............C=0.1, degree=2, gamma=1, kernel=rbf; total time=   8.8s
[CV] END ..............C=0.1, degree=2, gamma=1, kernel=poly; total time=   4.4s
[CV] END ..............C=0.1, degree=2, gamma=1, kernel=poly; total time=   4.4s
[CV] END ..............C=0.1, degree=2, gamma=1, kernel=poly; total time=   4.7s
[CV] END ..............C=0.1, degree=2, gamma=1, kernel=poly; total time=   4.6s
[CV] END ..............C=0.1, degree=2, gamma=1, kernel=poly; total time=   4.7s
[CV] END ...........C=0.1, degree=2, gamma=1, kernel=sigmoid; total time=   7.1s
[CV] END ...........C=0.1, degree=2, gamma=1, 

KeyboardInterrupt: 

In [347]:
df3.to_csv("churns.csv")

In [348]:
gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train_res, y_train_res)

cv(gb_clf,X_test_scaled,y_test)

accuracy: 0.8005521783181357
f1: 0.5867541278067593
roc_auc: 0.8422677681907371


In [370]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5, 7],
    'min_samples_split': [2, 4],
    'min_samples_leaf': [1, 2]
}

In [371]:
best_params = GridSearchCV(gb_clf, search_params, cv=5, verbose=True, n_jobs=-1).fit(X_train_res,y_train_res)

Fitting 5 folds for each of 320 candidates, totalling 1600 fits


In [374]:
best_params.best_params_

{'max_depth': 10,
 'max_features': 3,
 'min_samples_split': 2,
 'n_estimators': 100}

In [375]:
gb_clf.set_params(**best_params.best_params_)

GradientBoostingClassifier(max_depth=10, max_features=3)

In [376]:
cv(gb_clf,X_test_scaled,y_test)

accuracy: 0.7842350557244174
f1: 0.5511762063660658
roc_auc: 0.8284640429313399


In [377]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)

cv(xgb_clf,X_test_scaled,y_test)

accuracy: 0.7792755825734549
f1: 0.556599209833187
roc_auc: 0.8218812644943203


In [173]:
df = pd.read_csv("./churns.csv")

In [174]:
df.head()

,Unnamed: 0,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,InternetService_1,InternetService_2,Contract_1,Contract_2,PaymentMethod_1,PaymentMethod_2,PaymentMethod_3
0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,142,74,0,False,False,False,False,False,True,False
1,1,1,0,0,0,34,1,0,1,0,1,0,0,0,0,498,3624,0,False,False,True,False,False,False,True
2,2,1,0,0,0,2,1,0,1,1,0,0,0,0,1,436,536,1,False,False,False,False,False,False,True
3,3,1,0,0,0,45,0,0,1,0,1,1,0,0,0,266,3570,0,False,False,True,False,False,False,False
4,4,0,0,0,0,2,1,0,0,0,0,0,0,0,1,729,674,1,True,False,False,False,False,True,False


In [177]:
pd.to_numeric(df.TotalCharges,errors='coerce').isnull().sum()

0

In [178]:
df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [179]:
df = df[df.TotalCharges!=' ']

In [180]:
X = df.drop(["Churn"],axis=1)
y = df["Churn"]

In [181]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [182]:
sm = SMOTE(random_state=42)

X_train, y_train = sm.fit_resample(X_train, y_train)

In [183]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)

cv(xgb_clf,X_test,y_test)

accuracy: 0.7792755825734549
f1: 0.556599209833187
roc_auc: 0.8218812644943203


In [184]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,InternetService_1,InternetService_2,Contract_1,Contract_2,PaymentMethod_1,PaymentMethod_2,PaymentMethod_3
0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,142,74,0,False,False,False,False,False,True,False
1,1,0,0,0,34,1,0,1,0,1,0,0,0,0,498,3624,0,False,False,True,False,False,False,True
2,1,0,0,0,2,1,0,1,1,0,0,0,0,1,436,536,1,False,False,False,False,False,False,True
3,1,0,0,0,45,0,0,1,0,1,1,0,0,0,266,3570,0,False,False,True,False,False,False,False
4,0,0,0,0,2,1,0,0,0,0,0,0,0,1,729,674,1,True,False,False,False,False,True,False


In [185]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

predictions = log_reg.predict(X_test)

cv(log_reg,X_train,y_train)

accuracy: 0.8085055114522545
f1: 0.803952596326593
roc_auc: 0.8931367892102889


In [186]:
cv(log_reg,X_test,y_test)

accuracy: 0.8097720364741642
f1: 0.5978153189368652
roc_auc: 0.8564273910914176


In [188]:
clf = LGBMClassifier(n_estimators=100, learning_rate=0.1)
clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])
cv(clf, X_train, y_train)

[LightGBM] [Info] Number of positive: 4138, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000813 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 623
[LightGBM] [Info] Number of data points in the train set: 8276, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 3724, number of negative: 3724
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000701 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 623
[LightGBM] [Info] Number of data points in the train set: 7448, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[L

In [189]:
cv(clf, X_test,y_test)

[LightGBM] [Info] Number of positive: 335, number of negative: 933
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000251 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 623
[LightGBM] [Info] Number of data points in the train set: 1268, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.264196 -> initscore=-1.024275
[LightGBM] [Info] Start training from score -1.024275
[LightGBM] [Info] Number of positive: 335, number of negative: 933
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000145 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 623
[LightGBM] [Info] Number of data points in the train set: 1268, number of used features: 23
[LightGBM] [Info] [binary:Boos

In [54]:
X_train

,Unnamed: 0,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,InternetService_1,InternetService_2,Contract_1,Contract_2,PaymentMethod_1,PaymentMethod_2,PaymentMethod_3
2142,2142,0,0,0,1,21,1,0,1,0,1,0,0,1,0,625,2972,False,False,True,False,False,False,True
1623,1623,0,0,0,0,54,1,1,0,1,0,0,1,1,1,1230,5458,True,False,False,True,False,False,False
6074,6074,1,0,1,0,1,0,0,0,0,0,0,0,0,1,62,45,False,False,False,False,False,True,False
1362,1362,1,0,0,0,4,1,0,0,0,0,0,0,0,1,719,966,True,False,False,False,False,True,False
6754,6754,1,0,0,1,0,1,1,1,1,0,1,0,0,1,579,6530,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,3772,1,0,1,0,1,1,0,1,0,0,0,1,1,1,1188,486,True,False,False,False,False,True,False
5191,5191,0,0,1,1,23,1,1,1,1,1,1,1,1,1,1115,3848,False,False,False,True,True,False,False
5226,5226,1,0,1,1,12,1,0,0,0,0,0,0,0,1,52,1156,False,True,False,False,False,True,False
5390,5390,1,1,0,0,12,1,1,0,0,1,0,1,1,1,1270,2762,True,False,False,False,False,True,False


In [62]:
X_train.columns = [col.replace('{', '').replace('}', '').replace(':', '').replace(',', '').replace('&', '').replace('*', '').replace('#', '').replace('?', '').replace('|', '').replace("'", '').replace('"', '').replace('<', '').replace('>', '') for col in X_train.columns]
X_test.columns = [col.replace('{', '').replace('}', '').replace(':', '').replace(',', '').replace('&', '').replace('*', '').replace('#', '').replace('?', '').replace('|', '').replace("'", '').replace('"', '').replace('<', '').replace('>', '') for col in X_test.columns]


In [65]:
clf.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [66]:
help(clf)

Help on LGBMClassifier in module lightgbm.sklearn object:

class LGBMClassifier(sklearn.base.ClassifierMixin, LGBMModel)
 |  LGBMClassifier(boosting_type: str = 'gbdt', num_leaves: int = 31, max_depth: int = -1, learning_rate: float = 0.1, n_estimators: int = 100, subsample_for_bin: int = 200000, objective: Union[str, Callable[[Optional[numpy.ndarray], numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray]], Tuple[numpy.ndarray, numpy.ndarray]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray], Optional[numpy.ndarray]], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = None, class_weight: Union[Dict, str, NoneType] = None, min_split_gain: float = 0.0, min_child_weight: float = 0.001, min_child_samples: int = 20, subsample: float = 1.0, subsample_freq: int = 0, colsample_bytree: float = 1.0, reg_alpha: float = 0.0, reg_lambda: float = 0.0, random_state: Union[int, numpy.random.mtrand.Random

In [70]:
import optuna
# A tool for hyperparameter optimization

In [138]:
def objective(trial):
    # Define hyperparameters to tune
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 500),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-8, 1.0),
        'max_depth': trial.suggest_int('max_depth', -1, 50),
        # Include other parameters here
    }

    # Train your model
    clf = lgb.LGBMClassifier(**param)
    clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])
    
    # Predict and evaluate
    preds = clf.predict_proba(X_test)[:, 1]
    
    # Use a relevant metric for evaluation, here binary log loss for a binary classification problem
    loss = log_loss(y_test, preds)
    return loss


In [190]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2024-03-08 15:01:29,318] A new study created in memory with name: no-name-7a8dc3fc-644a-4c33-ae37-64fe92d44bf4


[LightGBM] [Info] Number of positive: 4138, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000813 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3294
[LightGBM] [Info] Number of data points in the train set: 8276, number of used features: 24
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


[I 2024-03-08 15:01:30,757] Trial 0 finished with value: -0.10835214446952596 and parameters: {'num_leaves': 93, 'min_data_in_leaf': 9, 'learning_rate': 1.773221247953648e-08, 'max_depth': 12, 'n_estimators': 448}. Best is trial 0 with value: -0.10835214446952596.


[LightGBM] [Info] Number of positive: 4138, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000913 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3294
[LightGBM] [Info] Number of data points in the train set: 8276, number of used features: 24
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

[I 2024-03-08 15:01:31,686] Trial 1 finished with value: -0.14736842105263157 and parameters: {'num_leaves': 166, 'min_data_in_leaf': 100, 'learning_rate': 2.5672592565084946e-07, 'max_depth': 14, 'n_estimators': 448}. Best is trial 1 with value: -0.14736842105263157.


...

[LightGBM] [Info] Number of positive: 4138, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000876 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3294
[LightGBM] [Info] Number of data points in the train set: 8276, number of used features: 24
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


[I 2024-03-08 15:02:46,050] Trial 98 finished with value: -0.12692307692307692 and parameters: {'num_leaves': 21, 'min_data_in_leaf': 72, 'learning_rate': 3.0341852726742438e-05, 'max_depth': 20, 'n_estimators': 383}. Best is trial 58 with value: -0.6119402985074627.


[LightGBM] [Info] Number of positive: 4138, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000746 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3294
[LightGBM] [Info] Number of data points in the train set: 8276, number of used features: 24
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


[I 2024-03-08 15:02:47,277] Trial 99 finished with value: -0.12692307692307692 and parameters: {'num_leaves': 46, 'min_data_in_leaf': 70, 'learning_rate': 1.5972026155336586e-05, 'max_depth': 14, 'n_estimators': 461}. Best is trial 58 with value: -0.6119402985074627.


Number of finished trials: 100
Best trial: {'num_leaves': 10, 'min_data_in_leaf': 76, 'learning_rate': 1.614012904027772e-06, 'max_depth': 9, 'n_estimators': 487}


In [191]:
clf.set_params(**study.best_trial.params)

LGBMClassifier(learning_rate=1.614012904027772e-06, max_depth=9,
               min_data_in_leaf=76, n_estimators=487, num_leaves=10)

In [192]:
clf.fit(X_train,y_train)
cv(clf,X_train,y_train)

[LightGBM] [Warning] min_data_in_leaf is set=76, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=76
[LightGBM] [Warning] min_data_in_leaf is set=76, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=76
[LightGBM] [Info] Number of positive: 4138, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000689 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 623
[LightGBM] [Info] Number of data points in the train set: 8276, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] min_data_in_leaf is set=76, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=76
[LightGBM] [Warning] min_data_in_leaf is set=76, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=76
[LightGBM] [I

In [193]:
cv(clf,X_test,y_test)

[LightGBM] [Warning] min_data_in_leaf is set=76, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=76
[LightGBM] [Warning] min_data_in_leaf is set=76, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=76
[LightGBM] [Info] Number of positive: 335, number of negative: 933
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000205 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 623
[LightGBM] [Info] Number of data points in the train set: 1268, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.264196 -> initscore=-1.024275
[LightGBM] [Info] Start training from score -1.024275
[LightGBM] [Warning] min_data_in_leaf is set=76, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=76
[LightGBM] [Warning] min_data_in_leaf is set=76, min_child_samples=20 will be igno

In [194]:
def objective_closure(X_train, y_train, X_test, y_test):
    def objective(trial):
        # Define the hyperparameters search space
        param = {
            'num_leaves': trial.suggest_int('num_leaves', 2, 256),
            'min_child_samples': trial.suggest_int('min_data_in_leaf', 5, 100),
            'learning_rate': trial.suggest_loguniform('learning_rate', 1e-8, 1.0),
            'max_depth': trial.suggest_int('max_depth', -1, 50),
            'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
            'objective': 'binary',
            'metric': 'binary_logloss',
            # Specify other parameters here
        }
        
        # Initialize and train the model
        clf = lgb.LGBMClassifier(**param)
        clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])
        
        # Predict on the validation set
        preds = clf.predict(X_test)
        
        # Calculate F1 score
        f1 = f1_score(y_test, preds)
        
        # Return the negative of the F1 score
        return f1

    return objective


In [195]:
objective = objective_closure(X_train, y_train, X_test, y_test)

# Create a study object specifying the optimization direction.
study = optuna.create_study(direction='maximize')  # Set direction to 'maximize'
study.optimize(objective, n_trials=100)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2024-03-08 15:04:25,190] A new study created in memory with name: no-name-02e64c5d-9167-4bbd-a17e-ef3117e1b5fe


[LightGBM] [Info] Number of positive: 4138, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000690 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 623
[LightGBM] [Info] Number of data points in the train set: 8276, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

[I 2024-03-08 15:04:26,484] Trial 0 finished with value: 0.5958005249343832 and parameters: {'num_leaves': 86, 'min_data_in_leaf': 55, 'learning_rate': 0.009234807858557606, 'max_depth': 20, 'n_estimators': 772}. Best is trial 0 with value: 0.5958005249343832.


[LightGBM] [Info] Number of positive: 4138, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000796 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 623
[LightGBM] [Info] Number of data points in the train set: 8276, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

[I 2024-03-08 15:04:28,070] Trial 1 finished with value: 0.6191646191646192 and parameters: {'num_leaves': 212, 'min_data_in_leaf': 41, 'learning_rate': 0.0010348980762450678, 'max_depth': 42, 'n_estimators': 571}. Best is trial 1 with value: 0.6191646191646192.


...

[LightGBM] [Info] Number of positive: 4138, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000795 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 623
[LightGBM] [Info] Number of data points in the train set: 8276, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


[I 2024-03-08 15:05:24,446] Trial 98 finished with value: 0.6431718061674009 and parameters: {'num_leaves': 14, 'min_data_in_leaf': 100, 'learning_rate': 0.02544577551753214, 'max_depth': 1, 'n_estimators': 968}. Best is trial 94 with value: 0.6520307354555434.


[LightGBM] [Info] Number of positive: 4138, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000821 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 623
[LightGBM] [Info] Number of data points in the train set: 8276, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


[I 2024-03-08 15:05:24,767] Trial 99 finished with value: 0.6437291897891232 and parameters: {'num_leaves': 145, 'min_data_in_leaf': 100, 'learning_rate': 0.031380095375342254, 'max_depth': 1, 'n_estimators': 970}. Best is trial 94 with value: 0.6520307354555434.


Number of finished trials: 100
Best trial: {'num_leaves': 15, 'min_data_in_leaf': 100, 'learning_rate': 0.0070961551558348605, 'max_depth': 2, 'n_estimators': 976}


In [196]:
clf.set_params(**study.best_trial.params)

LGBMClassifier(learning_rate=0.0070961551558348605, max_depth=2,
               min_data_in_leaf=100, n_estimators=976, num_leaves=15)

In [197]:
clf.fit(X_train,y_train)
cv(clf,X_train,y_train)

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Info] Number of positive: 4138, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000904 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 623
[LightGBM] [Info] Number of data points in the train set: 8276, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[Ligh

In [198]:
cv(clf,X_test,y_test)

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Info] Number of positive: 335, number of negative: 933
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000150 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 623
[LightGBM] [Info] Number of data points in the train set: 1268, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.264196 -> initscore=-1.024275
[LightGBM] [Info] Start training from score -1.024275
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

In [120]:
import tensorflow as tf
from tensorflow import keras

In [199]:
model = keras.Sequential([
    keras.layers.Dense(15, input_shape=(23,), activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [200]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).